<a href="https://colab.research.google.com/github/EllaScheltinga/Deep-Learning-Group-75/blob/main/AEGNN_NCARS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setting device and importing dependencies**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
#David
import sys
sys.path.append('/content/drive/MyDrive/Onderwijs/TU Delft: MSc RO/AEGNN/src')

In [ ]:
# Ella
import sys
sys.path.append('/content/drive/MyDrive/AEGNN/src')

In [ ]:
!pip install -q torch_geometric
!pip install pytorch-lightning
!pip install -q torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 884.9/884.9 kB 43.0 MB/s eta 0:00:00


In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### **Loading and preprocessing data**

In [ ]:
import tarfile
import os

# specify the path to the tar.gz file
tar_path = "/content/drive/MyDrive/Onderwijs/TU Delft: MSc RO/AEGNN/data/N-Cars_parsed.tar.gz"

# open the tar.gz file in read mode
tar = tarfile.open(tar_path, "r:gz")

# extract all files to a subdirectory in the content directory
tar.extractall(path=os.path.join(os.getcwd()))

# close the tar file
tar.close()

In [ ]:
import numpy as np
from torch_geometric.data import Data
import glob

def load_dat(raw_file: str) -> Data:  #From Git repository for N-cars
        events_file = os.path.join(raw_file, "events.txt")
        events = torch.from_numpy(np.loadtxt(events_file)).float().cuda()
        x, pos = events[:, -1:], events[:, :3]
        return Data(x=x, pos=pos)

def load_object(object_name):
  glob_ = glob.glob((os.path.join("/content/N-Cars_parsed/", object_name,'*')), recursive=True)
  data = []
  for item in glob_:
    data.append(load_dat(item))
  return data


In [ ]:
import torch

train = load_object('train')
validation = load_object('val')
test = load_object('test')

In [ ]:
def read_label(raw_file: str):
        label_file = os.path.join(raw_file, "is_car.txt")
        with open(label_file, "r") as f:
            label_txt = f.read().replace(" ", "").replace("\n", "")
        return "1" if label_txt == "1" else "0"

In [ ]:
from torch_geometric.transforms import FixedPoints
from torch_geometric.nn import radius_graph
from torch_geometric.transforms import Cartesian

def normalize_time(ts: torch.Tensor, beta: float = 0.5e-5):
    return (ts - torch.min(ts)) * beta

def sub_sampling(data: Data, n_samples: int, sub_sample: bool):
  
    sampler = FixedPoints(num=n_samples, allow_duplicates=False, replace=False)
    return sampler(data)

def pre_transform_one(data: Data, class_id):
        params = {"r": 3.0, "d_max": 32, "n_samples": 10000, "sampling": True}

        # Re-weight temporal vs. spatial dimensions to account for different resolutions.
        data.pos[:, 2] = normalize_time(data.pos[:, 2])

        # Coarsen graph by uniformly sampling n points from the event point cloud.
        data = sub_sampling(data, n_samples=params["n_samples"], sub_sample=params["sampling"])

        # Radius graph generation.
        data.edge_index = radius_graph(data.pos, r=params["r"], max_num_neighbors=params["d_max"])
        edge_attr = Cartesian(norm=True, cat=False, max_value=5.0)
        data.y = torch.tensor([class_id])
        data = edge_attr(data)
        return data

def pre_transform_all(data, class_id):
  pre_transformed_data = []

  for item in data:
    pre_transformed_data.append(pre_transform_one(item, class_id).to(device))

  return pre_transformed_data

SyntaxError: ignored

In [ ]:
import glob
import os

# glob_ = glob.glob(os.path.join("/content/N-Cars_parsed/*","*","is_car.txt"), recursive=True)

glob_test = glob.glob(os.path.join("/content/N-Cars_parsed/test/*"), recursive=True)
glob_train = glob.glob(os.path.join("/content/N-Cars_parsed/train/*"), recursive=True)
glob_val = glob.glob(os.path.join("/content/N-Cars_parsed/val/*"), recursive=True)
        
#1 is car, 0 is background
for item in glob_test:
  class_test  = int(read_label(item))
  
for item in glob_train:
  class_train  = int(read_label(item))

for item in glob_val:
  class_val  = int(read_label(item))

In [ ]:
class_test

0

In [ ]:
train = pre_transform_all(train, "train")
validation = pre_transform_all(validation, "validation")
test = pre_transform_all(test, "test")

In [ ]:
from torch.utils.data import random_split as Split
from torch_geometric.loader import DataLoader

BATCH_SIZE  = 10

# validation, test, train = Split(data, [0.1, 0.1, 0.8])
validation, test, train = DataLoader(validation, batch_size = BATCH_SIZE),\
                          DataLoader(test, batch_size = BATCH_SIZE),\
                          DataLoader(train, batch_size = BATCH_SIZE)

### **Training model**

In [ ]:
import torch.optim
from models.recognition import RecognitionModel

criterion = torch.nn.CrossEntropyLoss().cuda()
rm = RecognitionModel(network="graph_res", dataset="ncars", num_classes=2, img_shape=(100,100)).to(device)
optimizer = torch.optim.Adam(rm.parameters(), lr = 0.1)

In [ ]:
def training(model, data):
  seen = 0
  correct = 0
  for item in iter(data):
      item = item.to(device)
      optimizer.zero_grad()
      out = model.forward(item)   
      loss = criterion(out, item.y)
      loss.backward()
      pred = out.max(dim=1)[1]
      seen += len(item)
      correct += pred.eq(item.y).sum().item()
      optimizer.step()

  return correct / seen

In [ ]:
def validating(model, data):
    seen = 0
    correct = 0
    for item in iter(data):
        item = item.to(device)
        with torch.no_grad():
            out = model.forward(item)
            pred = out.max(dim=1)[1]
            seen += len(item)
            correct += pred.eq(item.y).sum().item()

    return correct / seen

In [ ]:
def testing(model, data):
    seen = 0
    correct = 0
    for item in iter(data):
        item = item.to(device)
        with torch.no_grad():
            out = model.forward(item)
            pred = out.max(dim=1)[1]
            seen += len(item)
            correct += pred.eq(item.y).sum().item()

    return correct / seen

In [ ]:
from tqdm import tqdm
import numpy as np

highest_acc = 0
num_epoch = 15
model = rm

av_hist = np.zeros((num_epoch, 3))
# writer = SummaryWriter()


for epoch in tqdm(range(num_epoch)):
  train_acc = training(model = model, data = train)
  validation_acc = validating(model = model, data = validation)

  av_hist[epoch]=np.array([epoch, train_acc, validation_acc])

  # Write metrics to Tensorboard
  # writer.add_scalar('Train Accuracy', train_acc , epoch)
  # writer.add_scalar('Validation Accuracy', validation_acc , epoch)


  if validation_acc > highest_acc:
      torch.save(model.state_dict(), 'best_model.pt')
      highest_acc = validation_acc

  print(f'Epoch {epoch:03d}, train acc.: {train_acc:.4f}, val. acc.: {validation_acc:.4f}',
                              flush = True)
print('\nFinished.')
# writer.flush()
# writer.close()

  0%|          | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch_geometric/utils/scatter.py:93: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(f"The usage of `scatter(reduce='{reduce}')` "
  0%|          | 0/15 [00:00<?, ?it/s]


AttributeError: ignored

### **Running model**

In [ ]:
# out = rm.forward(sample)
# pred = out.max(dim=1)[1]
# print(pred)